In [61]:
import os
from pathlib import Path
import pandas as pd
import json
import codecs
import numpy as np
import re
from collections import OrderedDict

In [59]:
raw_data_path_3 = '/home/pengfei/data/PDTB-3.0/data/raw/'
raw_data_path_2 = '/home/pengfei/data/2015-2016_conll_shared_task/raw'

In [60]:
folder_3 = os.listdir(raw_data_path_3)
folder_2 = os.listdir(raw_data_path_2)

In [61]:
file_3 = set([filename for folder in folder_3 for filename in os.listdir(raw_data_path_3 + folder)])

In [62]:
file_2 = set(folder_2)

In [63]:
len(file_2 - file_3)

0

In [64]:
len(file_3 - file_2)

399

In [66]:
len(file_2 - file_3)

0

# conclusion:
conll version does not contain 00, 01, and 24

In [8]:
gold_data_path_3 = '/home/pengfei/data/PDTB-3.0/data/gold/'
gold_data_path_2 = '/home/pengfei/data/2015-2016_conll_shared_task/data/conll16st-en-03-29-16-train/pdtb-parses.json'

In [9]:
folder_3 = os.listdir(gold_data_path_3)
file_3 = set([filename for folder in folder_3 for filename in os.listdir(gold_data_path_3 + folder)])
parse_dict = json.loads(codecs.open(gold_data_path_2, encoding='utf-8', errors='ignore').read())

In [73]:
file2 = set(parse_dict.keys())
file3 = file_3

# conclusion:
file3 contain files that is not in file2

In [76]:
set(o[4:6] for o in (file2 - file3))

set()

In [77]:
set(o[4:6] for o in (file3 - file2))

{'00', '01', '09', '12', '13', '22', '23', '24'}

In [82]:
for o in file3-file2:
    if o[4:6] in ['09', '12', '13']:
        print(o)

wsj_1344
wsj_0998
wsj_1202


## pdtb study

In [100]:
pdtb3 = pd.read_csv('../pdtb3.csv')
pdtb3.head()

,DocID,Relation_Type,Conn_SpanList,Conn_Src,Conn_Type,Conn_Pol,Conn_Det,Conn_Feat_SpanList,Conn1,SClass1A,...,Arg2_Det,Arg2_Feat_SpanList,Sup2_SpanList,Adju_Reason,Adju_Disagr,PB_Role,PB_Verb,Offset,Provenance,Link
0,wsj_0793,EntRel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,166,PDTB2::wsj_0793::166::SAME,NaN
1,wsj_0793,Explicit,197..204,NaN,NaN,NaN,NaN,NaN,instead,Expansion.Substitution.Arg2-as-subst,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,197..204,PDTB2::wsj_0793::197..204::CHANGED,NaN
2,wsj_0793,Implicit,NaN,NaN,NaN,NaN,NaN,NaN,then,Temporal.Asynchronous.Precedence,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,281,PDTB3,NaN
3,wsj_0793,Implicit,NaN,NaN,NaN,NaN,NaN,NaN,as a result,Contingency.Cause.Result,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,333,PDTB2::wsj_0793::333::SAME,NaN
4,wsj_0793,Implicit,NaN,NaN,NaN,NaN,NaN,NaN,but,Comparison.Concession.Arg2-as-denier,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,740,PDTB2::wsj_0793::740::CHANGED,NaN


In [112]:
Type = []
for i in range(len(pdtb3)):
    Type.append(pdtb3.loc[i, 'Relation_Type'])
print(set(Type)), 
print(pdtb3.keys())

{'Implicit', 'EntRel', 'NoRel', 'Hypophora', 'AltLexC', 'AltLex', 'Explicit'}
Index(['Relation_Type', 'Conn_SpanList', 'Conn_Src', 'Conn_Type', 'Conn_Pol',
       'Conn_Det', 'Conn_Feat_SpanList', 'Conn1', 'SClass1A', 'SClass1B',
       'Conn2', 'SClass2A', 'SClass2B', 'Sup1_SpanList', 'Arg1_SpanList',
       'Arg1_Src', 'Arg1_Type', 'Arg1_Pol', 'Arg1_Det', 'Arg2_Feat_SpanList',
       'Arg2_SpanList', 'Arg2_Src', 'Arg2_Type', 'Arg2_Pol', 'Arg2_Det',
       'Arg2_Feat_SpanList.1', 'Sup2_SpanList', 'Adju_Reason', 'Adju_Disagr',
       'PB_Role', 'PB_Verb', 'Offset', 'Provenance', 'Link'],
      dtype='object')


In [27]:
for i in range(len(pdtb3)):
    if pdtb3.loc[i, 'Relation_Type'] == 'Explicit':
        assert(type(pdtb3.loc[i, 'Conn_SpanList']) != float)
        assert(type(pdtb3.loc[i, 'Arg1_SpanList']) != float)
        assert(type(pdtb3.loc[i, 'Arg2_SpanList']) != float)
        assert(type(pdtb3.loc[i, 'Conn1']) != float)

In [25]:
for i in range(len(pdtb3)):
    if pdtb3.loc[i, 'Relation_Type'] == 'AltLex':
        assert(type(pdtb3.loc[i, 'Conn_SpanList']) != float)
        assert(type(pdtb3.loc[i, 'Conn1']) == float)

In [110]:
for i in range(len(pdtb3)):
    if pdtb3.loc[i, 'Relation_Type'] == 'AltLexC':
        assert(type(pdtb3.loc[i, 'Conn_SpanList']) != float)

In [99]:
for i in range(len(pdtb3)):
    if pdtb3.loc[i, 'Relation_Type'] == 'Implicit':
        assert(type(pdtb3.loc[i, 'Conn_SpanList']) == float)

In [101]:
for i in range(len(pdtb3)):
    if pdtb3.loc[i, 'Relation_Type'] == 'EntRel':
        assert(type(pdtb3.loc[i, 'Conn_SpanList']) == float)

In [105]:
for i in range(len(pdtb3)):
    if pdtb3.loc[i, 'Relation_Type'] == 'NoRel':
        assert(type(pdtb3.loc[i, 'Conn_SpanList']) == float)

In [107]:
for i in range(len(pdtb3)):
    if pdtb3.loc[i, 'Relation_Type'] == 'Hypophora':
        assert(type(pdtb3.loc[i, 'Conn_SpanList']) == float)

In [45]:
check = 'Link'
for i in range(len(pdtb3)):
    if not (type(pdtb3.loc[i, check]) == np.float64 or type(pdtb3.loc[i, check]) == float): 
        print(i, pdtb3.loc[i, check])
        break

17 LINK1


## for conn span list: number of uncorrected 4

In [49]:
main_folder = '/home/pengfei/data/PDTB-3.0/all/raw/'
false_conn_list = []
def get_span_list(span):
    if span == '':
        assert(False)
    spans = span.split(';')
    return [[int(k) for k in o.split('..')] for o in spans if o != '']


for i in range(len(pdtb3)):
    if pdtb3.loc[i, 'Relation_Type'] == 'Explicit':
        spanlist = get_span_list(pdtb3.loc[i, 'Conn_SpanList'])
        with open(main_folder + pdtb3.loc[i, 'DocID']) as f:
            rawtext = f.read()
            expected = ' '.join([rawtext[o[0]: o[1]] for o in spanlist]).lower()
        if pdtb3.loc[i, 'Conn1'] not in expected:
            print(pdtb3.loc[i, 'Conn1'], i, pdtb3.loc[i, 'DocID'], expected)
            false_conn_list.append(i)
            
print("Number of incorrect conn span list", len(false_conn_list))

# for i in false_conn_list:
#     with open(main_folder+pdtb3.loc[i,'DocID']) as f:
#         rawtext = f.read().lower()
#         start = int(pdtb3.loc[i, 'Conn_SpanList'].split('..')[0])
#         distance = [abs(m.start() - start) for m in re.finditer(pdtb3.loc[i,'Conn1'], rawtext, )]
#         start = [m.start() for m in re.finditer(pdtb3.loc[i, 'Conn1'], rawtext)]
#         if distance != []:
#             index = distance.index(min(distance))
#             start = start[index]
#             span = str(start) + '..' + str(start+len(pdtb3.loc[i, 'Conn1']))
#             pdtb3.loc[i, 'Conn_SpanList'] = span

till 1612 wsj_0771 'til
moreover 5315 wsj_0619 morever
because 51998 wsj_2215 'cause
moreover 52804 wsj_2202 morever
Number of incorrect conn span list 4


In [101]:
pdtb3.loc[41854,:]

DocID                                                       wsj_1617
Relation_Type                                               Explicit
Conn_SpanList                                             4330..4335
Conn_Src                                                         NaN
Conn_Type                                                        NaN
Conn_Pol                                                         NaN
Conn_Det                                                         NaN
Conn_Feat_SpanList                                               NaN
Conn1                                                           till
SClass1A              Contingency.Negative-condition.Arg2-as-negCond
SClass1B                                                         NaN
Conn2                                                            NaN
SClass2A                                                         NaN
SClass2B                                                         NaN
Sup1_SpanList                     

In [91]:
main_folder = '/home/pengfei/data/2015-2016_conll_shared_task/raw/'
with open(main_folder + pdtb3.loc[41854, 'DocID'], encoding='latin1') as f:
    rawtext = f.read()
rawtext[9: 14]

'After'

In [102]:
def get_doc_word_dict(parse_dict, DocID):
    ret={}
    doc_token_index = 0
    for sent_index, sentence in enumerate(parse_dict[DocID]['sentences']):
        for token_index, token in enumerate(sentence['words']):
            start = token[1]['CharacterOffsetBegin']
            end = token[1]['CharacterOffsetEnd']
            ret[(start, end)] = [start, end, doc_token_index, sent_index, token_index]
            doc_token_index += 1
    return ret
def get_token_list(char_span_list, doc_word_dict):
    tokenlist = []
    doc_word_dict = OrderedDict(sorted(doc_word_dict.items()), keys=lambda x:x[0][0])
    for span in char_span_list:
        for key, value in doc_word_dict.items():
            if type(key) == str:
                continue
            if key[1] > span[1]:
                break
            if key[0] >= span[0]:
                tokenlist.append(value)

    return tokenlist

doc_word_dict = get_doc_word_dict(parse_dict, 'wsj_1617')
get_token_list([[4330,4335]], doc_word_dict)

[[4330, 4335, 853, 35, 14]]

In [90]:
doc_word_dict

{(9, 14): [9, 14, 0, 0, 0],
 (15, 23): [15, 23, 1, 0, 1],
 (24, 32): [24, 32, 2, 0, 2],
 (33, 35): [33, 35, 3, 0, 3],
 (36, 39): [36, 39, 4, 0, 4],
 (40, 45): [40, 45, 5, 0, 5],
 (46, 53): [46, 53, 6, 0, 6],
 (54, 57): [54, 57, 7, 0, 7],
 (58, 63): [58, 63, 8, 0, 8],
 (63, 64): [63, 64, 9, 0, 9],
 (65, 68): [65, 68, 10, 0, 10],
 (69, 72): [69, 72, 11, 0, 11],
 (73, 81): [73, 81, 12, 0, 12],
 (82, 84): [82, 84, 13, 0, 13],
 (85, 91): [85, 91, 14, 0, 14],
 (92, 94): [92, 94, 15, 0, 15],
 (95, 99): [95, 99, 16, 0, 16],
 (100, 106): [100, 106, 17, 0, 17],
 (107, 108): [107, 108, 18, 0, 18],
 (109, 113): [109, 113, 19, 0, 19],
 (113, 114): [113, 114, 20, 0, 20],
 (117, 123): [117, 123, 21, 1, 0],
 (124, 127): [124, 127, 22, 1, 1],
 (128, 135): [128, 135, 23, 1, 2],
 (135, 136): [135, 136, 24, 1, 3],
 (137, 144): [137, 144, 25, 1, 4],
 (145, 148): [145, 148, 26, 1, 5],
 (149, 154): [149, 154, 27, 1, 6],
 (155, 165): [155, 165, 28, 1, 7],
 (166, 170): [166, 170, 29, 1, 8],
 (171, 180): [171, 

### check all the explicit connectives is in the connectives list
yes they are in the list

## check all raw data doesn't change

In [18]:
# pdtb2 vs pdtb3
main_folder = Path('/home/pengfei/data/pdtb_v2/all/raw')
filenames = [o for o in os.listdir(main_folder)]
pdtb2_raw_folder = main_folder
pdtb3_raw_folder = Path('/home/pengfei/data/PDTB-3.0/all/raw')

raw_false = []
for filename in filenames:
    with open(pdtb2_raw_folder/filename) as f:
        raw2 = f.read()
    with open(pdtb3_raw_folder/filename) as f:
        raw3 = f.read()
    if raw2 != raw3:
        raw_false.append(filename)
#         print(raw2.replace('\n', ''))
#         print()
#         print(raw3.replace('\n', ''))
#         print('\n\n\n')

## raw data does change adding space dot and \n

In [20]:
# conll vs pdtb3
main_folder = Path('/home/pengfei/data/2015-2016_conll_shared_task/raw')
filenames = [o for o in os.listdir(main_folder)]
pdtb2_raw_folder = main_folder
pdtb3_raw_folder = Path('/home/pengfei/data/PDTB-3.0/all/raw')

raw_false = []
for filename in filenames:
    raw2 = codecs.open(pdtb2_raw_folder/filename, encoding='utf-8',errors='ignore').read()
    if filename not in os.listdir(pdtb3_raw_folder):
        continue
    with open(pdtb3_raw_folder/filename) as f:
        raw3 = f.read()
    if raw2 != raw3:
        raw_false.append(filename)
#         print(raw2.replace('\n', ''))
#         print()
#         print(raw3.replace('\n', ''))
#         print('\n\n\n')

In [21]:
len(raw_false)

0

## for Arg1 spanlist

In [25]:
gold_data_path_2 = '/home/pengfei/data/2015-2016_conll_shared_task/data/conll16st-en-03-29-16-train/pdtb-parses.json'
parse_dict = json.loads(codecs.open(gold_data_path_2, encoding='utf-8', errors='ignore').read())
main_folder = '/home/pengfei/data/PDTB-3.0/all/raw/'

In [42]:
def get_list(parse_dict, doc):
    doc = parse_dict[doc]['sentences']
    char_start_list = []
    char_end_list = []
    for sentence in doc:
        for word in sentence['words']:
            char_start_list.append(word[1]['CharacterOffsetBegin'])
            char_end_list.append(word[1]['CharacterOffsetEnd'])
    return char_start_list, char_end_list


def get_span_string(span_list):
    ret = ''
    for span in span_list:
        ret += str(span[0])
        ret += '..'
        ret += str(span[1])
        ret += ';'
    return ret[:-1]

doc_list = []
for i in range(len(pdtb3)):
    if pdtb3.loc[i, 'Relation_Type'] == 'Explicit' and pdtb3.loc[i, 'DocID'] in parse_dict.keys():
        char_start_list, char_end_list = get_list(parse_dict, pdtb3.loc[i, 'DocID'])
        spanlist = get_span_list(pdtb3.loc[i, 'Arg1_SpanList'])
        for span in spanlist:
            if not ((span[0] in char_start_list) and (span[1] in char_end_list)):
                doc_list.append(min([abs(span[0]-o) for o in char_start_list]))

print(max(doc_list))

2


## the difference between corrected and not corrected

In [38]:
corrected = pd.read_csv('corrected.csv')
pdtb3 = pd.read_csv('pdtb3.csv')
for i in range(len(pdtb3)):
    if pdtb3.loc[i, 'Relation_Type'] == 'Explicit' and pdtb3.loc[i, 'DocID'] in parse_dict.keys():
        char_start_list, char_end_list = get_list(parse_dict, pdtb3.loc[i, 'DocID'])
        spanlist = get_span_list(pdtb3.loc[i, 'Arg1_SpanList'])
        for span in spanlist:
            if not ((span[0] in char_start_list) and (span[1] in char_end_list)):
                with open(main_folder  + pdtb3.loc[i, 'DocID']) as f:
                    rawtext = f.read()
                print(' '.join([rawtext[o[0]: o[1]] for o in spanlist]))
                print(' '.join([rawtext[o[0]: o[1]] for o in get_span_list(corrected.loc[i,'Arg1_SpanList'])]))
                print('\n\n\n')              
                

industrial buyers in Europe have the opportunity of getting reasonable prices in the U.S
industrial buyers in Europe have the opportunity of getting reasonable prices in the U.S.




If prices in the States go down, industrial buyers in Europe have the opportunity of getting reasonable prices in the U.S
If prices in the States go down, industrial buyers in Europe have the opportunity of getting reasonable prices in the U.S.




IMA Holdings Corp. completed its $3 billion acquisition of American Medical International Inc
IMA Holdings Corp. completed its $3 billion acquisition of American Medical International Inc.




A state trial judge in Illinois gave preliminary approval to a proposed settlement of a suit against a Bank of New York Co. unit, Irving Trust Co., over the interest rates on Irving's former One Wall Street Account money-market deposit accounts
state trial judge in Illinois gave preliminary approval to a proposed settlement of a suit against a Bank of New York Co. unit, Ir

You go to any medium-sized town in the U.S
You go to any medium-sized town in the U.S.




Assets of Construction Equipment International, Houston, were sold to Essex Crane Inc
Assets of Construction Equipment International, Houston, were sold to Essex Crane Inc.




the plant is scheduled to resume production on Nov. 20
the plant is scheduled to resume production on Nov. 20.




Colorado National is a unit of Colorado National Bankshares Inc
Colorado National is a unit of Colorado National Bankshares Inc.




British Aerospace PLC's Rover Group PLC subsidiary will also be able to offer the vehicles through its dealers in the U.K
British Aerospace PLC's Rover Group PLC subsidiary will also be able to offer the vehicles through its dealers in the U.K.




The young John Gutfreund had been discovered by Billy Salomon of Salomon Bros
The young John Gutfreund had been discovered by Billy Salomon of Salomon Bros.




Bank of New England Corp said it will sell some operations and lay off 4% 

A permanent smoking ban on virtually all domestic airline routes won approval from the House
permanent smoking ban on virtually all domestic airline routes won approval from the House




A permanent smoking ban on virtually all domestic airline routes won approval from the House, which separately sent to President Bush a nearly $67 billion fiscal 1990 bill including the first construction funds for the space station
permanent smoking ban on virtually all domestic airline routes won approval from the House, which separately sent to President Bush a nearly $67 billion fiscal 1990 bill including the first construction funds for the space station




Sen. Pete Domenici (R., N.M.), the ranking Republican on the Senate Budget Committee, used his influence to preserve more than $132,000 in subsidies for air service to Sante Fe, N.M
Sen. Pete Domenici (R., N.M.), the ranking Republican on the Senate Budget Committee, used his influence to preserve more than $132,000 in subsidies for air servi

## for arg2 spanlist

In [112]:
def get_list(parse_dict, doc):
    doc = parse_dict[doc]['sentences']
    char_start_list = []
    char_end_list = []
    for sentence in doc:
        for word in sentence['words']:
            char_start_list.append(word[1]['CharacterOffsetBegin'])
            char_end_list.append(word[1]['CharacterOffsetEnd'])
    return char_start_list, char_end_list

count = 0
for i in range(len(pdtb3)):
    if pdtb3.loc[i, 'Relation_Type'] == 'Explicit' and pdtb3.loc[i, 'DocID'] in parse_dict.keys():
        char_start_list, char_end_list = get_list(parse_dict, pdtb3.loc[i, 'DocID'])
        spanlist = get_span_list(pdtb3.loc[i, 'Arg2_SpanList'])
        for span in spanlist:
            if not ((span[0] in char_start_list) and (span[1] in char_end_list)):
                count += 1
#                 print(i, pdtb3.loc[i, 'DocID'], spanlist)

print("the number of Arg1 error", count)

the number of Arg1 error 77


In [43]:
doc_list = []
for i in range(len(pdtb3)):
    if pdtb3.loc[i, 'Relation_Type'] == 'Explicit' and pdtb3.loc[i, 'DocID'] in parse_dict.keys():
        char_start_list, char_end_list = get_list(parse_dict, pdtb3.loc[i, 'DocID'])
        spanlist = get_span_list(pdtb3.loc[i, 'Arg2_SpanList'])
        for span in spanlist:
            if not ((span[0] in char_start_list) and (span[1] in char_end_list)):
                doc_list.append(min([abs(span[0]-o) for o in char_start_list]))

print(max(doc_list))

2


## for pdtb2 arg1

In [132]:
pdtb2 = pd.read_csv('/home/pengfei/Github/pdtb2/pdtb2.csv', low_memory=False)
gold_data_path_2 = '/home/pengfei/data/2015-2016_conll_shared_task/data/conll16st-en-03-29-16-train/pdtb-parses.json'
parse_dict = json.loads(codecs.open(gold_data_path_2, encoding='utf-8', errors='ignore').read())
def get_list(parse_dict, doc):
    doc = parse_dict[doc]['sentences']
    char_start_list = []
    char_end_list = []
    for sentence in doc:
        for word in sentence['words']:
            char_start_list.append(word[1]['CharacterOffsetBegin'])
            char_end_list.append(word[1]['CharacterOffsetEnd'])
    return char_start_list, char_end_list

def get_docid(pdtb2, i):
    return 'wsj_' + '{:02}'.format(pdtb2.loc[i, 'Section']) + '{:02}'.format(pdtb2.loc[i, 'FileNumber'])

count = 0
for i in range(len(pdtb2)):
    if pdtb2.loc[i, 'Relation'] == 'Explicit' and get_docid(pdtb2, i) in parse_dict.keys():
        char_start_list, char_end_list = get_list(parse_dict, get_docid(pdtb2, i))
        spanlist = get_span_list(pdtb3.loc[i, 'Arg1_SpanList'])
        for span in spanlist:
            if not ((span[0] in char_start_list) and (span[1] in char_end_list)):
                count += 1
print("the number of Arg1 error", count)     

the number of Arg1 error 14940


## the use of 'Offset' is not clear

## the use of 'Link' is not clear

In [43]:
main_folder = '/home/pengfei/data/PDTB-3.0/all/raw/'

In [38]:
count = 0
for i in range(len(pdtb3)):
    pass
#     if type(pdtb3.loc[i, 'Offset']) != float and pdtb3.loc[i, 'Relation_Type'] == 'Explicit':
#         if pdtb3.loc[i, 'Offset'] not in pdtb3.loc[i, 'Conn_SpanList']:
#             print(pdtb3.loc[i, 'Offset'], pdtb3.loc[i, 'Conn_SpanList'])